In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier as knn
import random
from time import process_time

#### Data Preprocessing

In [2]:
df = pd.read_csv("sobar-72.csv")

In [3]:
df.head()

,behavior_sexualRisk,behavior_eating,behavior_personalHygine,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,ca_cervix
0,10,13,12,4,7,9,10,1,8,7,3,14,8,5,7,12,12,11,8,1
1,10,11,11,10,14,7,7,5,5,4,2,15,13,7,6,5,5,4,4,1
2,10,15,3,2,14,8,10,1,4,7,2,7,3,3,6,11,3,3,15,1
3,10,11,10,10,15,7,7,1,5,4,2,15,13,7,4,4,4,4,4,1
4,8,11,7,8,10,7,8,1,5,3,2,15,5,3,6,12,5,4,7,1


In [4]:
df.describe()

,behavior_sexualRisk,behavior_eating,behavior_personalHygine,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,ca_cervix
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,9.666667,12.791667,11.083333,7.902778,13.347222,7.180556,8.611111,3.125000,8.486111,8.513889,5.388889,12.652778,9.694444,8.097222,6.166667,10.375000,10.541667,9.319444,10.277778,0.291667
std,1.186782,2.361293,3.033847,2.738148,2.374511,1.522844,1.515698,1.845722,4.907577,4.275686,3.400727,3.207209,4.130406,4.243171,2.897303,4.316485,4.366768,4.181874,4.482273,0.457719
min,2.000000,3.000000,3.000000,2.000000,6.000000,2.000000,4.000000,1.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,0.000000
25%,10.000000,11.000000,9.000000,6.000000,11.000000,6.000000,8.000000,1.000000,3.000000,5.000000,2.000000,11.000000,7.000000,3.000000,3.750000,6.750000,7.000000,5.000000,6.750000,0.000000
50%,10.000000,13.000000,11.000000,10.000000,15.000000,7.000000,9.000000,3.000000,7.000000,8.000000,4.000000,14.000000,11.000000,9.000000,6.500000,12.000000,12.000000,10.000000,11.000000,0.000000
75%,10.000000,15.000000,14.000000,10.000000,15.000000,8.000000,10.000000,5.000000,14.000000,13.000000,9.000000,15.000000,13.000000,11.250000,9.000000,14.250000,15.000000,13.000000,15.000000,1.000000
max,10.000000,15.000000,15.000000,10.000000,15.000000,10.000000,10.000000,5.000000,15.000000,15.000000,10.000000,15.000000,15.000000,15.000000,10.000000,15.000000,15.000000,15.000000,15.000000,1.000000


In [5]:
rows,cols = df.shape

In [6]:
columns = df.columns

In [7]:
columns = columns.drop(labels = "ca_cervix")

In [8]:
df_std = pd.DataFrame()
for i in columns:
    df_std.loc[:,i] = (df.loc[:,i]-df.loc[:,i].mean())/df.loc[:,i].std()
df_std.insert(column = "ca_cervix", value = df.iloc[:,-1], loc = df_std.shape[1])

In [9]:
df_std.head()

,behavior_sexualRisk,behavior_eating,behavior_personalHygine,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,ca_cervix
0,0.280872,0.088229,0.302147,-1.425335,-2.673065,1.194768,0.916336,-1.151311,-0.099053,-0.354069,-0.702464,0.420061,-0.410237,-0.729931,0.287624,0.376464,0.333962,0.401867,-0.508175,1
1,0.280872,-0.758765,-0.027468,0.765927,0.274910,-0.118565,-1.062950,1.015863,-0.710353,-1.055711,-0.996519,0.731858,0.800298,-0.258585,-0.057525,-1.245226,-1.269055,-1.272024,-1.400579,1
2,0.280872,0.935222,-2.664384,-2.155755,0.274910,0.538101,0.916336,-1.151311,-0.914119,-0.354069,-0.996519,-1.762522,-1.620772,-1.201277,-0.057525,0.144794,-1.727059,-1.511151,1.053533,1
3,0.280872,-0.758765,-0.357082,0.765927,0.696050,-0.118565,-1.062950,-1.151311,-0.710353,-1.055711,-0.996519,0.731858,0.800298,-0.258585,-0.747822,-1.476896,-1.498057,-1.272024,-1.400579,1
4,-1.404358,-0.758765,-1.345926,0.035507,-1.409647,-0.118565,-0.403188,-1.151311,-0.710353,-1.289592,-0.996519,0.731858,-1.136558,-1.201277,-0.057525,0.376464,-1.269055,-1.272024,-0.731276,1


In [10]:
X_train = df_std.iloc[:rows-15,:-2]
X_test = df_std.iloc[rows-15:, :-2]
y_train = df_std.iloc[:rows-15,-1]
y_test = df_std.iloc[rows-15:, -1]

In [11]:
y_train.shape

(57,)

#### cNN model application

In [12]:
#eucledian distance
def dist(x,y):
    return np.sqrt(np.dot(x-y,x-y))           

In [13]:
#subset for training examples
list_s = []

In [14]:
def cNN(X,y):
    #initialize
    s = []
    init = np.random.choice(X.index)
    s.append(init)
    itr = 0

    #break the loop when no transfer occurs 
    while(True):
        model = knn(n_neighbors = 1)
        temp_X = X.loc[s]
        temp_y = y.loc[s]
        model.fit(temp_X,temp_y)
        flag = True
        for i in X_train.index:
            res = model.predict(X.loc[[i]])
            if(res != y.loc[i]):
                s.append(i)
                flag = False
                break
        if(flag):
            break
        itr+=1
        
    print("Total {0} number of training iterations".format(itr))
    return s

In [15]:
subset = cNN(X_train,y_train)

Total 19 number of training iterations


In [16]:
print(subset) #prints the index values of data points in the subset
X_train_sub = X_train.loc[subset]
y_train_sub = y_train.loc[subset]

[18, 21, 0, 1, 5, 7, 8, 13, 19, 29, 31, 35, 36, 37, 41, 20, 49, 14, 52, 53]


#### Testing accuracy for cNN

In [17]:
model = knn(n_neighbors=5)
model.fit(X_train_sub, y_train_sub)
start = process_time()
cNN_score = model.score(X_test,y_test)
end = process_time()

In [18]:
print("training subset size")
X_train.loc[subset].shape

training subset size


(20, 18)

In [19]:
print("Accuracy of cNN model is {0}% and inference time on training set is {1} sec".format(cNN_score*100, end-start))

Accuracy of cNN model is 73.33333333333333% and inference time on training set is 0.004757999999999818 sec


#### Comparing with standard KNN for K = 5

In [20]:
print("training set size")
X_train.shape

training set size


(57, 18)

In [21]:
model = knn(n_neighbors=5)
model.fit(X_train,y_train)
start = process_time()
kNN_score = model.score(X_test,y_test)
end = process_time()

In [22]:
print("Accuracy of KNN model is {0}% and inference time on training set is {1} sec".format(kNN_score*100, end-start))

Accuracy of KNN model is 73.33333333333333% and inference time on training set is 0.0027620000000001532 sec
